In [1]:
import numpy as np
from itertools import permutations
from tqdm import tqdm
import itertools

In [2]:
def checkConstraints(value, constraints):
    
    for constraint in constraints:
        if value in constraints[constraint][0] or value in constraints[constraint][1]:
            return True
    return False  # didnt pass any of the constraints

In [3]:
def checkOrder(constraints, ticket):  # and again by the order of the Peaky Blinders xDD

    pbar = tqdm(total=value-1)
    element_orders = []
    for possible_order in permutations(constraints):
        check_order = []
        for order_element in list(possible_order):
            for value in ticket:
                if value in constraints[order_element][0] or value in constraints[order_element][1]:
                    check_order.append("MATCH")
                else:
                    check_order.append("NOT MATCH")
        shape = (len(constraints),len(constraints))
        check = np.diag(np.array(check_order).reshape(shape))
        
        if np.all(check[0] == check):  # all elements are the same
            element_orders.append(possible_order)
    return element_orders
     

In [4]:
def checkValid(tickets, constraints):

    valid_tickets = []

    for ticket in tickets:
        for value in ticket:
            if checkConstraints(value, constraints):
                continue
            else:
                break

        else:  # only executed if the inner loop did NOT break
            valid_tickets.append(ticket)
    return valid_tickets


In [5]:
def findOrder(valid_tickets, constraints):  # by the order of the peaky blinders xD
    orders = []
    for ticket in valid_tickets:
        print(ticket)
        correct_order = checkOrder(constraints, ticket)
        orders.append(correct_order)
        return correct_order


In [6]:
with open('input.txt') as f:
    all_lines = [line.strip() for line in f.readlines()]

    constraint_names = [str([i for i in constraint.split(": ")][0]) for constraint in all_lines[:20]]
    constraint_ranges = [[range(int(condition[0]), int(condition[1])) for condition in [val.split("-") for val in [i for i in constraint.split(": ")][1].split(" or ")]] for constraint in all_lines[:20]]
    constraints = dict(zip(constraint_names, constraint_ranges))
    
    my_ticket = [int(val) for val in all_lines[22:23][0].split(",")]
    nearby_tickets = [[int(val) for val in ticket.split(",")] for ticket in all_lines[25:]]
    valid_tickets = checkValid(nearby_tickets, constraints)

    #correct_order = findOrder(valid_tickets, constraints)

In [7]:
with open('test_input.txt') as f:
    all_lines = [line.strip() for line in f.readlines()]

    constraint_names = [str([i for i in constraint.split(": ")][0]) for constraint in all_lines[:3]]
    constraint_ranges = [[range(int(condition[0]), int(condition[1])+ 1) for condition in [val.split("-") for val in [i for i in constraint.split(": ")][1].split(" or ")]] for constraint in all_lines[:3]]
    constraints = dict(zip(constraint_names, constraint_ranges))
    
    
    my_ticket = [int(val) for val in all_lines[5:6][0].split(",")]
    nearby_tickets = [[int(val) for val in ticket.split(",")] for ticket in all_lines[8:]]
    valid_tickets = checkValid(nearby_tickets, constraints)
    
    correct_order = findOrder(valid_tickets, constraints)
    

[7, 3, 47]


UnboundLocalError: local variable 'value' referenced before assignment

In [111]:
def innerShuffle(valid_tickets, constraints, adj_constraints):

    #print("\n\tBegin innerShuffle")
    all_arrays = []  # will become multidimensional list
    
    #print("\tAdj Constraints:", adj_constraints)
    #print("\n\t Tickets loop")
    for ticket in valid_tickets:
        #print("\t Ticket:", ticket)
        # list with all constraints if they are satiefied, otherwise 0 for constraints that are not 
        tmp_array = [key_c  if val in constraints[key_c][0] or val in constraints[key_c][1] else 0 for key_c, val in zip(adj_constraints, ticket)]
        #print("\tConstraints & Ticket:\n\t\t", tmp_array)
        all_arrays.append(tmp_array)  # adding each list
    #print("\n\t Out of ticket loop.")
    shuffle_elem=[]
    shuffle_ind=[]
    all_arrays=np.array(all_arrays)
    bool_cols = np.all(all_arrays == all_arrays[0,:], axis = 0)  # true values for all cols that are same
    #print("\n\tTrue columns:", bool_cols)
    
    #print("\n\tBool Loop")
    shuffle_sorted = []
    for ind,val in enumerate(bool_cols):

        adj_c= adj_constraints  # ?
        #print("\tBool value:", val)
        if val == True:
            #print("\tAdding to shuffle_sort:", adj_c[ind])
            shuffle_sorted.append(adj_c[ind])  # adding the element

        else: # False
            #print(adj_c)
            #print("\tAdding to shuffle_sort:", adj_c[ind])
            shuffle_sorted.append(adj_c[ind])  # adding the element

            shuffle_elem.append(adj_c[ind])
            shuffle_ind.append(ind)  # saving the index of the non matching ones
        
        #print("\n\n InnerShuffle wrong index:", shuffle_ind)
    
    return shuffle_sorted, shuffle_elem, shuffle_ind, bool_cols

In [127]:
def shuffleSort(valid_tickets, constraints):

    shuffleSorted = False

    #constraints = [i for i in constraints.keys()]
    adj_constraints = list(constraints.keys())  # starting order of constraints

    while shuffleSorted == False:
        # print("Main Loop Starts\n\n")
        # print("Initializing innerShuffle.")
        shuffle_sorted = innerShuffle(valid_tickets, constraints, adj_constraints)
        # print("\nFinished innerShuffle.")
        # print("\n Wrong index", shuffle_sorted[2])
        # print("\n shuffle_sorted:", shuffle_sorted[1])

        combs = list(itertools.product(shuffle_sorted[2], shuffle_sorted[1]))
        
        print("\n Combinations loop")
        for comb in combs:
            print("\n\tCombination:",comb)
            #print(shuffle_sorted[0])
            shuffle_sorted[0][comb[0]] = comb[1]
            print("\n\tSo far:",shuffle_sorted[1])
            print("\n\tWrong:", len(shuffle_sorted[2]))
            shuffle_sorted = innerShuffle(valid_tickets, constraints, shuffle_sorted[0])

            if len(shuffle_sorted[2]) == 0:
                return shuffle_sorted
            #print(shuffle_sorted[3])
            #break
        print(shuffle_sorted)
        break



In [128]:
x= shuffleSort(valid_tickets, constraints)


 Combinations loop

	Combination: (0, 'departure location')

	So far: ['departure location', 'departure date', 'arrival location', 'arrival station', 'arrival platform', 'duration', 'price', 'type', 'zone']

	Wrong: 9

	Combination: (0, 'departure date')

	So far: ['departure location', 'departure date', 'arrival location', 'arrival station', 'arrival platform', 'duration', 'price', 'type', 'zone']

	Wrong: 9

	Combination: (0, 'arrival location')

	So far: ['departure date', 'departure date', 'arrival location', 'arrival station', 'arrival platform', 'duration', 'price', 'type', 'zone']

	Wrong: 9

	Combination: (0, 'arrival station')

	So far: ['arrival location', 'departure date', 'arrival location', 'arrival station', 'arrival platform', 'duration', 'price', 'type', 'zone']

	Wrong: 9

	Combination: (0, 'arrival platform')

	So far: ['arrival station', 'departure date', 'arrival location', 'arrival station', 'arrival platform', 'duration', 'price', 'type', 'zone']

	Wrong: 9

	Com